##### FIGER: 
    subfolder:FIGER; 
    original data: train.json, test.json;
    formatted data: train_200k_v2.json, val_partitioned_v2.json, test_lines.json 
        train_200k_v2.json is a sample of train.json, train.json has 3M sentences
    ontology: ontology.txt, the taxonomy is trivial (follow the types' names)

##### Ontonotes: 
    subfolder:ontonotes; 
    original data: g_train_tree.json, g_dev_tree.json g_test_tree.json;
    ontology: entity_typing_all_datasets/data/ontology/onto_ontology.txt, the taxonomy is trivial (follow the types' names)

##### Balanced Ontonotes: 
    subfolder:balanced_ontonotes; 
    original data: train.json, dev.json test.json;
    ontology: entity_typing_all_datasets/data/ontology/onto_ontology.txt, the taxonomy is trivial (follow the types' names)

##### Choi: 
    subfolder:crowd; 
    original data: train_m_tree.json, dev_tree.json test_tree.json;
    ask to Manuel (è dispersa tra i meandri del codice di choi/lopez)

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
bbn_path = base_dir + "FIGER/"

Mounted at /content/drive/


In [2]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

with open(bbn_path + "test.json", 'r') as inp:
    examples_test = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  1505765 elementi


In [4]:
print("il dataset test contiene ",len(examples_test), "elementi")

il dataset test contiene  434 elementi


In [10]:
import itertools
phrase = []
phrases = []
permutations = []
num_total_phrases = 0
for i in range(0, 4):
  all_list = []
  if len(entire_dataset_list[i]['mentions'])>1:

    # creo tutte le permutazioni
    for j in entire_dataset_list[i]['mentions']:
      all_list.append(j['labels'])
    permutations = list(itertools.product(*all_list))

    num_total_phrases += len(permutations)

    # creo tutte le possibili frasi a partire dalle permutazioni
    for p in permutations:
      phrase = entire_dataset_list[i]['tokens'].copy()
      for h in range(0, len(p)):
        phrase[entire_dataset_list[i]['mentions'][h]['start']] = p[h]
      
      phrases.append(phrase)
    
  else:
    for k in entire_dataset_list[i]['mentions'][0]['labels']:
      phrase = entire_dataset_list[i]['tokens'].copy()
      phrase[entire_dataset_list[i]['mentions'][0]['start']] = k
      phrases.append(phrase)
      num_total_phrases+=1

print("numero totale di frasi", num_total_phrases)
print("controllo se il numero di frasi totali è corretto", num_total_phrases == len(phrases))
print(phrases)

numero totale di frasi 51
controllo se il numero di frasi totali è corretto True
[['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/person/musician', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/person/musician', 'Soundsystem', 'and', 'written', 'music', 'for', '/person/musician', ')', '.'], ['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/person/musician', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/person/musician', 'Soundsystem', 'and', 'written', 'music', 'for', '/person/artist', ')', '.'], ['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/person/musician', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/person/musician', 'Soundsystem', 'and', 'written', 'music', 'for', '/person', ')', '.'], ['The', 'band', 'also', 'shared', '

Merge the two dataset train e test

In [5]:
entire_dataset_list = examples_train + examples_test
print(" uniti train e test contengono ", len(entire_dataset_list), "elementi")

 uniti train e test contengono  1506199 elementi


In [6]:
print(entire_dataset_list[0])

{'tokens': ['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', 'Out', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', 'LCD', 'Soundsystem', 'and', 'written', 'music', 'for', 'Cake', ')', '.'], 'senid': 25, 'mentions': [{'start': 11, 'labels': ['/person/musician', '/person/artist', '/person'], 'end': 13}, {'start': 22, 'labels': ['/person/musician', '/person/artist', '/person'], 'end': 24}, {'start': 28, 'labels': ['/person/musician', '/person/artist', '/person'], 'end': 29}], 'fileid': 'wex/20110513/0/0/5'}


In [ ]:
with open(bbn_path + "corpus_data.txt", 'w') as f:
    for s in phrases:
        f.write(str(s) + '\n')